<a href="https://colab.research.google.com/github/ScriptA01/-telescopio_spectrum/blob/main/telesc%C3%B3pio_Spectrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sejam bem vindos a física de particulas**

Nesta tarefa, vamos usar um conjunto de dados da Nasa, mais especificamente do telescópio Spectrum onde temos informações que determinam se um registro se trata de *raio Gama (um fóton, emitido a partir da fusão de proton e neutron que geraram um dêuteron)* ou *Hadron (particulas que interagem por meio de força nuclear)* .  

A entrega deverá ser individual e utilizando Colab, não esqueça do seu nome e RA na entrega e de compartilhar comigo seu colab. 

PS: Não se assustem com a complexidade do dado, façam o que aprenderam em aula.

Infelizmente a única referencia de valor que encontrei foi em inglês:  https://www.sciencedirect.com/science/article/abs/pii/S0168900222004247

# **Integrantes do time**

Nome: Nicolas Bernardo Gonçalves de Souza



Nome: Leonardo Ferreira Mendes



Nome: Marcio Roberto de Souza Filho


Nome: Alessandro Silva



Nome: Guilherme Mario Macedo de Assis



In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
import matplotlib.pyplot as plt
import seaborn as sns


auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [ ]:
# Baixa os dados de treinamento
link = "https://drive.google.com/file/d/12NghHZhyGTmxFOnRSHKZr44zj2uhISu1/view?usp=share_link"
id = link.split("/")[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('telescope.csv')
df = pd.read_csv('telescope.csv')
df.shape

(19020, 11)

In [ ]:
df.sample(5) #Verifica os cabeçalhos para saber se tenho que transformar variáveis objeto em categórica

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
7104,32.1014,22.8106,3.1340,0.2203,0.1201,5.1226,-7.4837,12.1902,4.9770,155.6500,g
10896,43.4800,8.6907,2.5347,0.3971,0.2117,42.5859,-22.1743,-5.9437,1.7010,180.0130,g
9715,154.0370,17.3139,2.9292,0.2719,0.1560,113.9150,118.9650,-12.6074,50.7380,71.8962,g
17239,35.1379,6.4970,2.7193,0.5954,0.3273,26.7800,21.5213,-8.2411,46.0560,234.2070,h
2336,25.1407,10.8048,2.3865,0.5626,0.3593,4.0279,17.9199,10.9738,2.4381,171.2120,g


In [ ]:
df.columns = ['fLength', 'fWidth', 'fSize',	'fConc',	'fConc1',	'fAsym',	'fM3Long',	'fM3Trans',	'fAlpha',	'fDist' ,'classific'] #A coluna class estava dando erro de sintaxe então troquei o nome dela

In [ ]:
df.classific.value_counts() #Verificando quantos valores tenho de cada para depois definir qual virou 1 e qual virou 0

g    12332
h     6688
Name: classific, dtype: int64

In [ ]:
objlist = df.select_dtypes(include='object').columns #Seleciono as colunas objeto

In [ ]:
for i in objlist:
  df[i] = df[i].astype('category')
  df[i] = df[i].cat.codes

#Transformo colunas objeto em categóricas

In [ ]:
df.sample(5) #Verifico como ficaram os valores

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,classific
6162,32.8666,17.7862,2.3683,0.4797,0.2420,-29.6960,-9.0224,12.1372,58.6010,68.4377,0
13350,60.8079,18.2385,2.9456,0.3161,0.1690,60.9532,59.0673,-16.1833,2.6888,242.2668,1
4277,23.8603,14.8311,2.4224,0.5104,0.2703,-9.5178,-12.7220,12.4322,0.5570,98.4508,0
11666,20.2153,12.2239,2.3170,0.4964,0.2964,0.3888,3.5360,10.8678,39.3447,57.3510,0
14183,85.5400,37.1281,3.3141,0.1669,0.0879,-2.9802,61.0820,30.6979,70.2363,148.2932,1


In [ ]:
df.classific.value_counts() #Verificando quantos valores tenho de cada para depois definir qual virou 1 e qual virou 0

0    12332
1     6688
Name: classific, dtype: int64

Documentando quais valores foram trocados
*   0 = g = gama (sinal)
*   1 = h = hádron (fundo)




Treinamento do modelo

In [ ]:
X = df.drop('classific', axis=1) # Retirando a variável target do modelo de treinamento
y = df['classific']

In [ ]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import RandomizedSearchCV

ABORDAGEM I

In [ ]:
# Divisão de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=987)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree

In [ ]:
#Definição de parâmetros do modelo
rf_model = RandomForestClassifier(n_estimators=200, 
                                  max_depth=4, 
                                  n_jobs=-1, 
                                  min_samples_split=2, 
                                  min_samples_leaf=2,
                                  max_features=None)

In [ ]:
#Treinando o modelo
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
#Verificando os principais KPIs para entender o quão efetivo é o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8259726603575184
Precision: 0.8408485104957961
Recall: 0.774643014785245
F1-score: 0.792456550476369


ABORDAGEM II

Diminui o tamanho do teste para ter mais dados de treino e aumentei a quantidade de estimativas para que tenha um leque maior em que averiguar as predições

In [ ]:
# Divisão de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=987)

In [ ]:
#Definição de parâmetros do modelo
rf_model = RandomForestClassifier(n_estimators=300, 
                                  max_depth=4, 
                                  n_jobs=-1, 
                                  min_samples_split=2, 
                                  min_samples_leaf=2,
                                  max_features=None)

In [ ]:
#Treinando o modelo
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
#Verificando os principais KPIs para entender o quão efetivo é o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.82465825446898
Precision: 0.8438321988597215
Recall: 0.7729775602231745
F1-score: 0.7909767341393679


ABORDAGEM III

Diminui mais ainda o numero de teste para ter mais dados para treino e alterei a quantidade de aleatoriedade dos pontos no modelo, como temos 19.020 linhas, utilizei o número 190 pensando em proporção numérica. Aumentei o número de estimativas e a quantidade de níveis que a árvore vai descer, para que tenhamos mais testes e com uma aleatoriedade mais controlada dentro do tamanho do modelo, tomando bastante cuidado com overfitting.

In [ ]:
# Divisão de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=190)

In [ ]:
#Definição de parâmetros do modelo
rf_model = RandomForestClassifier(n_estimators=500, 
                                  max_depth=5, 
                                  n_jobs=-1, 
                                  min_samples_split=2, 
                                  min_samples_leaf=2,
                                  max_features=None)

In [ ]:
#Treinando o modelo
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
#Verificando os principais KPIs para entender o quão efetivo é o modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8391167192429022
Precision: 0.8478375155470643
Recall: 0.7951789166466171
F1-score: 0.8115014153296065


CONCLUSÃO
---
Percebo que os modelos precisam ter dados consideráveis para teste, por conta disso sempre que movimento a quantidade de teste acaba que os KPIs de predição do modelo melhoram, neste em específico movimentei também a quantidade de níveis que a árvore pode descer e números de estimativas que não influenciaram tanto, tudo ficou mais dentro da quantidade de teste que utilizo e a aleatoriedade dos pontos definidos do modelo com o random_state